In [1]:
import numpy as np
import glob, os
import random as rd
from keras import layers as ly
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam, SGD
from keras.layers import Input, Lambda
from keras import losses
from keras.models import model_from_json
import keras.backend.tensorflow_backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
import string, json
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './REAL/seq/*'
alpha = list(string.ascii_uppercase)
chr2index = {'C':1, 'N':2, 'O':3, 'F':4, 'P':5, 'S':6, 'Cl':7, 'As': 8, 'I':9, 'Hg':10}
#chr2index = {'H':1, 'B':2,  'C':3, 'N':4, 'O':5, 'F':6, 'P':7, 'S':8, 'Cl':9, 'Fe':10,  'Co':11, 'Cu':12, 'Zn':13, 'Br':14}
#chr2index = {'H':1, 'B':5,  'C':6, 'N':7, 'O':8, 'F':9, 'P':15, 'S':16, 'Cl':17, 'Fe':26,  'Co':27, 'Cu':29, 'Zn':30, 'Br':35}

In [3]:
chr2index

{'C': 1,
 'N': 2,
 'O': 3,
 'F': 4,
 'P': 5,
 'S': 6,
 'Cl': 7,
 'As': 8,
 'I': 9,
 'Hg': 10}

In [4]:
def chr2OH(node):
    oh = [0 for i in range(len(chr2index))]
    index = chr2index[node]-1
    oh[index] = 1
    return oh

In [5]:
train_label = [[8, 9, 10, 5, 7, 6, 16, 2, 1, 15, 13, 14, 20, 4], [16, 13, 3, 15, 2, 17, 19, 8, 11, 6, 12, 9, 18, 20], [16, 4, 2, 14, 19, 1, 17, 3, 20, 12, 9, 8, 10, 7], [2, 5, 1, 4, 7, 18, 8, 11, 20, 17, 15, 12, 16, 19], [15, 14, 13, 5, 4, 3, 9, 18, 7, 20, 8, 11, 16, 1]]
val_label = [[11, 17], [7, 1], [13, 5], [14, 3], [6, 12]]
test_label = [[12, 3, 19, 18], [10, 4, 14, 5], [6, 11, 15, 18], [13, 10, 6, 9], [17, 10, 19, 2]]

In [6]:
# file read
all_names = []
all_data = []
#test = set()
sequence_length = []
alpha = list(string.ascii_uppercase)
data_length = len(glob.glob(dir))
file_predix = './REAL/seq/'
for index in range(data_length):
    filename = file_predix + str(index) + "/"
    prefix = str(index) + 'REAL'
    print(filename)
    nums = [i for i in range(1, 21)]
    rd.shuffle(nums)
    print('x_train', train_label[index], 'x_val', val_label[index], 'x_test', test_label[index])
    x_train = []
    x_val = []
    for num in train_label[index]:
        f = filename + prefix + str(num) + '-*'
        files = glob.glob(f)
        for file in files:
            dts = []
            for rf in open(file, 'r'):
                (u, v, w) = rf[1:-2].split(', ')
                #dts.append([chr2index[u[1:-1]], chr2index[v[1:-1]], float(w)])
                dts.append(chr2OH(u[1:-1]) + chr2OH(v[1:-1]) +[float(w)])
            sequence_length.append(len(dts))
            x_train.append(dts)
            
    for num in val_label[index]:
        f = filename + prefix + str(num) + '-*'
        files = glob.glob(f)
        for file in files:
            dts = []
            for rf in open(file, 'r'):
                (u, v, w) = rf[1:-2].split(', ')
                dts.append(chr2OH(u[1:-1]) + chr2OH(v[1:-1]) +[float(w)])
                #dts.append([chr2index[u[1:-1]], chr2index[v[1:-1]], float(w)])
            sequence_length.append(len(dts))
            x_val.append(dts)
            
x_train = np.array([np.array(arr) for arr in x_train])
x_val = np.array([np.array(arr) for arr in x_val])
'''# file read
all_names = []
all_data = []
#test = set()
sequence_length = []
alpha = list(string.ascii_uppercase)
data_length = len(glob.glob(dir))
file_predix = './REAL/seq/'
for index in range(data_length):
    filename = file_predix + str(index) + "/"
    prefix = str(index) + 'REAL'
    print(filename)
    nums = [i for i in range(1, 21)]
    rd.shuffle(nums)
    print('x_train', nums[:14], 'x_val', nums[14:16], 'x_test', nums[16:])
    x_train = []
    x_val = []
    for num in nums[:14]:
        f = filename + prefix + str(num) + '-*'
        files = glob.glob(f)
        for file in files:
            dts = []
            for rf in open(file, 'r'):
                (u, v, w) = rf[1:-2].split(', ')
                dts.append(chr2OH(u[1:-1]) + chr2OH(v[1:-1]) +[float(w)])
            sequence_length.append(len(dts))
            x_train.append(dts)
            
    for num in nums[14:16]:
        f = filename + prefix + str(num) + '-*'
        files = glob.glob(f)
        for file in files:
            dts = []
            for rf in open(file, 'r'):
                (u, v, w) = rf[1:-2].split(', ')
                dts.append(chr2OH(u[1:-1]) + chr2OH(v[1:-1]) +[float(w)])
            sequence_length.append(len(dts))
            x_val.append(dts)
            
x_train = np.array([np.array(arr) for arr in x_train])
x_val = np.array([np.array(arr) for arr in x_val])
'''

./REAL/seq/0/
x_train [8, 9, 10, 5, 7, 6, 16, 2, 1, 15, 13, 14, 20, 4] x_val [11, 17] x_test [12, 3, 19, 18]
./REAL/seq/1/
x_train [16, 13, 3, 15, 2, 17, 19, 8, 11, 6, 12, 9, 18, 20] x_val [7, 1] x_test [10, 4, 14, 5]
./REAL/seq/2/
x_train [16, 4, 2, 14, 19, 1, 17, 3, 20, 12, 9, 8, 10, 7] x_val [13, 5] x_test [6, 11, 15, 18]
./REAL/seq/3/
x_train [2, 5, 1, 4, 7, 18, 8, 11, 20, 17, 15, 12, 16, 19] x_val [14, 3] x_test [13, 10, 6, 9]
./REAL/seq/4/
x_train [15, 14, 13, 5, 4, 3, 9, 18, 7, 20, 8, 11, 16, 1] x_val [6, 12] x_test [17, 10, 19, 2]


'# file read\nall_names = []\nall_data = []\n#test = set()\nsequence_length = []\nalpha = list(string.ascii_uppercase)\ndata_length = len(glob.glob(dir))\nfile_predix = \'./REAL/seq/\'\nfor index in range(data_length):\n    filename = file_predix + str(index) + "/"\n    prefix = str(index) + \'REAL\'\n    print(filename)\n    nums = [i for i in range(1, 21)]\n    rd.shuffle(nums)\n    print(\'x_train\', nums[:14], \'x_val\', nums[14:16], \'x_test\', nums[16:])\n    x_train = []\n    x_val = []\n    for num in nums[:14]:\n        f = filename + prefix + str(num) + \'-*\'\n        files = glob.glob(f)\n        for file in files:\n            dts = []\n            for rf in open(file, \'r\'):\n                (u, v, w) = rf[1:-2].split(\', \')\n                dts.append(chr2OH(u[1:-1]) + chr2OH(v[1:-1]) +[float(w)])\n            sequence_length.append(len(dts))\n            x_train.append(dts)\n            \n    for num in nums[14:16]:\n        f = filename + prefix + str(num) + \'-*\'\n

In [7]:
print(len(x_train), len(x_val))
#max_sequence_length

659 98


In [10]:
#x_train, x_test, train_name, test_name = train_test_split(all_data, all_names, test_size=0.3)
#x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [11]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 200
steps_per_epoch = len(x_train)

In [12]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.kld(y_true, y_pred) # categorical_crossentropy
    return loss1 * 0.7 + loss2 * 0.3

def custom_loss1(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.categorical_crossentropy(y_true, y_pred) # categorical_crossentropy
    return loss1 * 0.7 + loss2 * 0.3

In [13]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 21))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(21))(decoded)

W0528 02:40:25.568122 139759467755264 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0528 02:40:25.576388 139759467755264 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0528 02:40:25.578418 139759467755264 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [14]:
def val_generator(x_val):
    idx = 0
    while True:
        yield np.array([x_val[idx]]), np.array([x_val[idx]])
        idx += 1
        if idx >= len(x_val):
            idx = 0

def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx += 1
        if idx >= len(x_train):
            idx = 0

In [17]:
rd.shuffle(x_train)
rd.shuffle(x_val)

In [26]:
kld_ae = Model(inputs, decoded)
kld_ae.compile(loss=custom_loss, optimizer='adam')#decay=0.9
kld_hist = kld_ae.fit_generator(train_generator(x_train), epochs=200, steps_per_epoch=steps_per_epoch, verbose=1, validation_steps=len(x_val), validation_data=val_generator(x_val))

Epoch 1/200
659/659 [==============================] - 36s 55ms/step - loss: 0.1348 - val_loss: 0.1451
Epoch 2/200
659/659 [==============================] - 33s 50ms/step - loss: 0.1306 - val_loss: 0.1750
Epoch 3/200
659/659 [==============================] - 32s 48ms/step - loss: 0.1307 - val_loss: 0.1396
Epoch 4/200
659/659 [==============================] - 32s 49ms/step - loss: 0.1282 - val_loss: 0.1376
Epoch 5/200
659/659 [==============================] - 32s 49ms/step - loss: 0.1278 - val_loss: 0.1351
Epoch 6/200
659/659 [==============================] - 32s 48ms/step - loss: 0.1268 - val_loss: 0.1360
Epoch 7/200
659/659 [==============================] - 32s 48ms/step - loss: 0.1267 - val_loss: 0.1332
Epoch 8/200
659/659 [==============================] - 32s 48ms/step - loss: 0.1239 - val_loss: 0.1275
Epoch 9/200
659/659 [==============================] - 32s 48ms/step - loss: 0.1221 - val_loss: 0.1271
Epoch 10/200
659/659 [==============================] - 32s 48ms/step - l

Epoch 80/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0753 - val_loss: 0.0726
Epoch 81/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0968 - val_loss: 0.1129
Epoch 82/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0955 - val_loss: 0.0939
Epoch 83/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0852 - val_loss: 0.0855
Epoch 84/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0810 - val_loss: 0.0822
Epoch 85/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0771 - val_loss: 0.0768
Epoch 86/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0767 - val_loss: 0.0781
Epoch 87/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0752 - val_loss: 0.0744
Epoch 88/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0726 - val_loss: 0.0742
Epoch 89/200
659/659 [==============================] - 31s 47ms

659/659 [==============================] - 31s 47ms/step - loss: 0.0694 - val_loss: 0.0856
Epoch 159/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0698 - val_loss: 0.0631
Epoch 160/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0617 - val_loss: 0.0619
Epoch 161/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0593 - val_loss: 0.0589
Epoch 162/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0576 - val_loss: 0.0642
Epoch 163/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0624 - val_loss: 0.0694
Epoch 164/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0633 - val_loss: 0.0632
Epoch 165/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0627 - val_loss: 0.0744
Epoch 166/200
659/659 [==============================] - 31s 47ms/step - loss: 0.0734 - val_loss: 0.0684
Epoch 167/200
659/659 [==============================] - 31s 47ms/ste

In [27]:
# MODEL SAVE
model_json = kld_ae.to_json()
filename = 'REAL_kld_real_model' #input('filename: ') 
with open('models/real/2REAL_kld_real_model.json', 'w') as file:
    file.write(model_json)
kld_ae.save_weights('models/real/2weights_' +  filename + '.h5')


with open('models/real/2REAL_kld_real_history.json', 'w') as f:
    json.dump(kld_hist.history, f)

In [35]:
len(x_train)

1377

In [23]:
kld_ae1 = Model(inputs, decoded)
kld_ae1.compile(loss=custom_loss, optimizer='adam')#decay=0.9
kld_hist = kld_ae1.fit_generator(train_generator(x_train), epochs=200, steps_per_epoch=steps_per_epoch, verbose=1, validation_steps=len(x_val), validation_data=val_generator(x_val))

Epoch 1/200
640/659 [============================>.] - ETA: 0s - loss: 0.1342

KeyboardInterrupt: 

In [ ]:
# MODEL SAVE
model_json = kld_ae1.to_json()
filename = '2REAL_kld_real_model' #input('filename: ') 
with open('models/real/2REAL_kld_real_model.json', 'w') as file:
    file.write(model_json)
kld_ae.save_weights('models/real/2weights_' +  filename + '.h5')


with open('models/real/2REAL_kld_real_history.json', 'w') as f:
    json.dump(kld_hist.history, f)

In [23]:
model_path = 'models/real/chr_weights' + '{epoch:02d}-{loss:.4f}.h5'
checkpoint_callback = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only = True, save_weights_only = True, period=100)

In [25]:
mse_ae = Model(inputs, decoded)
mse_ae.compile(loss='mse', optimizer='adam')#decay=0.9
mse_hist = mse_ae.fit_generator(train_generator(x_train), epochs=1, steps_per_epoch=steps_per_epoch, verbose=1, validation_steps=len(x_val), validation_data=val_generator(x_val))

Epoch 1/1
659/659 [==============================] - 35s 53ms/step - loss: 0.0401 - val_loss: 0.0408


In [ ]:
cce_inputs = Input(shape=(None, 17))
cce_encoded = LSTM(128, return_sequences=True)(cce_inputs)  #activation 안적으면 tanh
cce_encoded = LSTM(64, activation='relu')(cce_encoded)

cce_decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([cce_encoded, cce_inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

cce_decoded = LSTM(64, return_sequences=True)(cce_decoded)
cce_decoded = LSTM(128, return_sequences=True)(cce_decoded)
cce_decoded = TimeDistributed(Dense(17))(cce_decoded)

In [13]:
cce_ae = Model(inputs, decoded)
cce_ae.compile(loss=custom_loss1, optimizer='adam')#Adam(lr=0.0002, decay=1e-3))#Adam(lr=0.003))#decay=0.9
cce_hist = cce_ae.fit_generator(train_generator(x_train), epochs=200, steps_per_epoch=steps_per_epoch, verbose=1, validation_steps=len(x_val), validation_data=val_generator(x_val))
#cce_hist = cce_ae.fit_generator(train_generator(x_train), epochs=200, steps_per_epoch=steps_per_epoch, verbose=1, validation_steps=len(x_val), validation_data=val_generator(x_val))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/200
4563/4563 [==============================] - 107s 23ms/step - loss: 4.7056 - val_loss: 1.2168
Epoch 2/200
4563/4563 [==============================] - 105s 23ms/step - loss: 3.3900 - val_loss: 1.2168
Epoch 3/200
4563/4563 [==============================] - 105s 23ms/step - loss: 2.9607 - val_loss: 1.2159
Epoch 4/200
4563/4563 [==============================] - 105s 23ms/step - loss: 2.8799 - val_loss: 15.3959
Epoch 5/200
4563/4563 [==============================] - 102s 22ms/step - loss: 2.7521 - val_loss: 1.2159
Epoch 6/200
4563/4563 [==============================] - 102s 22ms/step - loss: 2.4007 - val_loss: 1.8026
Epoch 7/200
4563/4563 [==============================] - 102s 22ms/step - loss: 2.2163 - val_loss: 1.2154
Epoch 8/200
4563/4563 [==============================] - 102s 22ms/step - loss: 2.3507 - val_loss: 1.2161
Epoch 9/200
4563/4563 [==============================]

4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 72/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 73/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 74/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 75/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 76/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 77/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 78/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 79/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epoch 80/200
4563/4563 [==============================] - 100s 22ms/step - loss: nan - val_loss: nan
Epo